# Histogram Equalization

Load raw video and apply histogram equalization to improve contrast

In [1]:
from LeptonUtils import *
import numpy as np
import time
import cv2

In [3]:
# Convert BGR imgage to 1-channel, 16-bit image
# (B, G, R) = [upper byte, lower byte, 0]
def bgr2raw16(bgr_frame):
    return np.left_shift((bgr_frame[:,:,0].astype("uint16")), 8) + (bgr_frame[:,:,1].astype("uint16"))

In [42]:
# ===== Set Temp. Range (Celsius) =====
min_temp = f2c(62)
max_temp = f2c(67)

In [1]:
# Load video (BGR encoding)
# video_capture = cv2.VideoCapture(
#     filename="vids/gas_on.mkv",
#     apiPreference=cv2.CAP_FFMPEG
# )
# assert video_capture.isOpened(), "Error reading video file"

# Load video (.tiff)
video_capture = Raw16Video("./vids/Shiny_Facing_Cam.tiff")

# Create video writer object to save processed video
video_writer = cv2.VideoWriter(
    filename=f"./out/rec_{int(time.time())}.mp4",
    apiPreference=cv2.CAP_FFMPEG,
    fourcc=cv2.VideoWriter_fourcc(*'XVID'),
    fps=8.0,
    frameSize=(160, 120),
    params=[
        cv2.VIDEOWRITER_PROP_IS_COLOR, 0
    ],
)
assert video_writer.isOpened()

# Display processed video and save frames to file
cv2.namedWindow("recording", cv2.WINDOW_NORMAL)
while cv2.waitKey(1) & 0xFF != ord('q'):
    start = time.time()
    
    # Grab frame
    ret, frame = video_capture.read()
    if not ret: break

    # Process frame
    # frame = bgr2raw16(frame)
    frame = clip_norm(frame, temp2raw(min_temp), temp2raw(max_temp))
    # frame = hist_equalize(frame, True)

    # Append frame to video
    video_writer.write(frame)
    cv2.imshow('recording', frame)

    # Pause video
    if cv2.waitKey(1) & 0xFF == ord('p'):
        time.sleep(1)
        cv2.imwrite("frame.tiff", frame) 
        while cv2.waitKey(1) & 0xFF != ord('p'): time.sleep(0.1)
    else:
        # Play frames at original fps
        time.sleep(max(0, 1/8-(time.time()-start))) 

video_writer.release()
cv2.destroyAllWindows()

NameError: name 'Raw16Video' is not defined